In [11]:
import pandas as pd
import numpy as np
import bqplot as bq
import ipywidgets as widgets

# Load CO₂ emissions per capita data (from OWID)
url = "https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv"
df = pd.read_csv(url)

# Focus on a few countries and years
countries = ['United States', 'China', 'India', 'Germany']
df = df[df['country'].isin(countries) & df['year'].between(1960, 2020)]
df = df[['country', 'year', 'co2_per_capita']].dropna()

# Structure data for animation
years = sorted(df['year'].unique())
co2_data = {
    year: df[df['year'] == year].set_index('country').reindex(countries)['co2_per_capita'].fillna(0).values
    for year in years
}

# Create scales
x_sc = bq.OrdinalScale()
y_sc = bq.LinearScale()

# Initial bars
bar = bq.Bars(x=countries, y=co2_data[years[0]], scales={'x': x_sc, 'y': y_sc})

# Create axes and figure
x_ax = bq.Axis(scale=x_sc, label='Country')
y_ax = bq.Axis(scale=y_sc, orientation='vertical', label='CO₂ per Capita (tons)')

fig = bq.Figure(marks=[bar], axes=[x_ax, y_ax], title=f'CO₂ per Capita - {years[0]}')

# Controls
slider = widgets.IntSlider(min=0, max=len(years) - 1, step=1, description='Year')
play = widgets.Play(min=0, max=len(years) - 1, interval=300)
widgets.jslink((play, 'value'), (slider, 'value'))

# Update function
def update(change):
    i = change['new']
    year = years[i]
    bar.y = co2_data[year]
    fig.title = f'CO₂ per Capita - {year}'

slider.observe(update, names='value')

# Display
widgets.VBox([fig, widgets.HBox([play, slider])])
